# A Machine Learning Approach to Reducing Missed Appointments to Improve health efficiency and Resource Utilisation

Objective:
To reduce missed medical appointments (no-shows) by predicting patient attendance 
using SQL-based data preprocessing, GitHub version control, Tableau for visual insight, 
and Python machine learning models.

In [ ]:
# Import Libraries

# Core libraries for data manipulation and operations
import pandas as pd                      # For data loading, manipulation, and analysis
import numpy as np                       # For numerical operations and array handling
from collections import Counter          # For counting class distributions

# Visualization libraries
import matplotlib.pyplot as plt          # For creating static, animated, and interactive plots
import seaborn as sns                    # For enhanced data visualization (built on top of matplotlib)
sns.set(style='whitegrid')               # Use a clean white grid style for all seaborn plots

# SHAP for model explainability
import shap                              # For interpreting model predictions (especially tree-based models)

# Scikit-learn: model selection, preprocessing, and pipeline management
from sklearn.model_selection import (
    train_test_split,                    # To split the dataset into training and testing sets
    GridSearchCV,                        # For hyperparameter tuning using cross-validation
    StratifiedKFold                      # For stratified cross-validation
)
from sklearn.preprocessing import StandardScaler     # For scaling numeric features to standard normal
from sklearn.pipeline import Pipeline                 # For creating streamlined ML workflows

# Classification performance metrics
from sklearn.metrics import (
    accuracy_score,                      # Overall accuracy of predictions
    balanced_accuracy_score,             # Adjusted accuracy score for imbalanced datasets
    precision_score,                     # Precision: proportion of true positives among predicted positives
    recall_score,                        # Recall: proportion of true positives among actual positives
    f1_score,                            # Harmonic mean of precision and recall
    roc_auc_score,                       # Area under the ROC curve
    roc_curve,                           # ROC curve values (TPR, FPR)
    classification_report,              # Complete performance summary (precision, recall, f1-score)
    confusion_matrix,                   # Matrix of actual vs predicted classes
    ConfusionMatrixDisplay              # Visual display of confusion matrix
)

# Classification models
from sklearn.linear_model import LogisticRegression         # Logistic Regression classifier
from sklearn.tree import DecisionTreeClassifier             # Decision Tree classifier
from sklearn.ensemble import RandomForestClassifier         # Random Forest classifier (ensemble of trees)
from xgboost import XGBClassifier                           # Extreme Gradient Boosting (XGBoost) classifier

# Sampling techniques for handling class imbalance
from imblearn.over_sampling import SMOTE                    # Synthetic Minority Over-sampling Technique
from imblearn.under_sampling import (
    RandomUnderSampler,                                     # Randomly under-samples majority class
    NeighbourhoodCleaningRule                               # Removes ambiguous/noisy instances from majority class
)
from imblearn.pipeline import Pipeline as ImbPipeline       # Pipeline combining sampling and modeling (avoid conflict with sklearn)

In [ ]:
# Load and preview the dataset
df = pd.read_csv('Patient_Appointment_Attendance.csv')
print("First 5 rows of the dataset:")
print(df.head())

# Display dataset information
print("\nDataset info:")
print(df.info())

In [ ]:
# Check for duplicate rows
duplicate_count = df.duplicated().sum()
print(f"\nNumber of duplicate rows: {duplicate_count}")

# Check for missing values in each column
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0]
print("\nMissing values in each column (if any):")
print(missing_values)

# Check number of unique values for each column
unique_values = df.nunique().sort_values(ascending=False)
print("\nUnique values in each column:")
print(unique_values)

In [ ]:
# Examine unique values in 'Handcap' column before transformation
unique_handcap_values = df['Handcap'].value_counts().sort_index()
print("\nUnique values in 'Handcap' column before binarization:")
print(unique_handcap_values)

# Binarize 'Handcap': 0 for no disability, 1 for any level of disability
df['Handcap'] = df['Handcap'].apply(lambda x: 1 if x > 0 else 0)

# Confirm the changes to 'Handcap'
handcap_binarized_counts = df['Handcap'].value_counts()
print("\nCounts of binarized 'Handcap' values:")
print(handcap_binarized_counts)

In [ ]:
# Examine unique values in 'Age' column before transformation
unique_age_values = df['Age'].value_counts().sort_index()
print("\nUnique values in 'Age' column before binarization:")
print(unique_age_values)

# Drop rows with invalid age values
df = df[df['Age'] >= 0]

# Confirm the update
print("Minimum age after cleaning:", df['Age'].min())
print("Maximum age after cleaning:", df['Age'].max())

In [ ]:
# Encode 'Gender' as binary: 0 for Female, 1 for Male
df['Gender'] = df['Gender'].map({'F': 0, 'M': 1})
print("\nUnique values in 'Gender' after encoding:")
print(df['Gender'].value_counts())

# Convert date columns to datetime objects with timezone awareness
df['ScheduledDay'] = pd.to_datetime(df['ScheduledDay'], utc=True)
df['AppointmentDay'] = pd.to_datetime(df['AppointmentDay'], utc=True)

# Create 'LeadTime' feature: days between scheduling and appointment
df['LeadTime'] = (df['AppointmentDay'] - df['ScheduledDay']).dt.days

# Extract day of the week for the appointment
df['AppointmentWeekday'] = df['AppointmentDay'].dt.dayofweek  # Monday=0, Sunday=6

# Encode 'No-show' as binary: 0 = showed up, 1 = no-show
df['No_show_Int'] = df['No-show'].map({'No': 0, 'Yes': 1})
print("\nEncoded 'No_show_Int' column value counts:")
print(df['No_show_Int'].value_counts())

In [ ]:
# Examine unique values in 'LeadTime' column before transformation
unique_leadtime_values = df['LeadTime'].value_counts().sort_index()
print("\nUnique values in 'LeadTime' column before binarization:")
print(unique_leadtime_values)

In [ ]:
print("First 5 rows of the dataset:")
print(df.head())

# Display dataset information
print("\nDataset info:")
print(df.info())

In [ ]:
# Filter rows with negative LeadTime
negative_leadtime_rows = df[df['LeadTime'] < 0]

# Print the number of rows with negative LeadTime
print("Number of rows with negative LeadTime:", negative_leadtime_rows.shape[0])

# Print the first 5 rows with negative LeadTime for inspection
print("\nSample rows with negative LeadTime:")
print(negative_leadtime_rows.head())


It was observed that some values in the `LeadTime` column were negative. 
Further inspection revealed that in these cases, the `ScheduledDay` was recorded as occurring after the `AppointmentDay`, 
which is logically incorrect. Specifically, entries with a `LeadTime` of -1 corresponded to same-day scheduling and appointment. 
These values were therefore recoded to 0. 
Rows with `LeadTime` values less than -1 were considered erroneous and removed from the dataset, 
as they were both few in number and unlikely to contribute meaningfully to the model.

In [ ]:
# Convert LeadTime == -1 to 0 (same-day scheduling and appointment)
df.loc[df['LeadTime'] == -1, 'LeadTime'] = 0

# Drop rows with LeadTime < -1 (invalid scheduling)
df = df[df['LeadTime'] >= -1]  # this now drops only rows less than -1

# Confirm the update
print("Number of rows after cleaning LeadTime:", df.shape[0])

In [ ]:
print("Last 5 rows of the dataset:")
print(df.tail())

# Display dataset information
print("\nDataset info:")
print(df.info())

In [ ]:
# Save the biased dataset
df.to_csv("Preprocessed_patient_appointment.csv", index=False)

In [ ]:
# Drop unnecessary columns
df.drop(columns=["AppointmentID", "PatientId", "ScheduledDay", "AppointmentDay", "No-show", "AppointmentWeekday"], inplace=True)

# Confirm remaining columns
print("\nRemaining columns after drop:")
print(df.columns)

print("Last 5 rows of the dataset:")
print(df.tail())

# Display dataset information
print("\nDataset info:")
print(df.info())

In [ ]:
# Select numeric columns only
numeric_df = df.select_dtypes(include='number')

# Compute correlation matrix
corr_matrix = numeric_df.corr()

# Plot the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)

# Add title and layout
plt.title("Correlation Heatmap (Numeric Features Only)")
plt.tight_layout()
plt.show()

In [ ]:
# Get value counts
no_show_counts = df['No_show_Int'].value_counts().sort_index()  # 0: showed up, 1: no-show

# Define labels
labels = ['Showed Up', 'No-Show']

# Define colors (optional)
colors = ['#66b3ff', '#ff9999']

# Plot pie chart
plt.figure(figsize=(6, 6))
plt.pie(no_show_counts, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
plt.title('Patient Appointment Attendance Distribution')
plt.axis('equal')  # Equal aspect ratio ensures the pie chart is a circle
plt.show()

It was observed that the dataset was imbalanced, with a significant bias toward the majority class — patients who showed up for 
their appointments. This imbalance posed a risk of skewing the machine learning models toward favouring the majority class, 
potentially reducing their ability to accurately predict no-show instances. 
To address this, bias-mitigation techniques such as resampling methods were employed 
to ensure fairer and more reliable model performance across both classes.

In [ ]:
# Create a random sample of 20000 rows from the original DataFrame
df_sample = df.sample(n=20000, random_state=42)  # random_state for reproducibility


# Features and target
X = df_sample.drop(columns=['No_show_Int'])
y = df_sample['No_show_Int']


# Identify categorical columns
categorical_cols = X.select_dtypes(include=['object', 'category']).columns

# One-hot encode categorical columns
X = pd.get_dummies(X, columns=categorical_cols, drop_first=True)
X.head()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import warnings

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, balanced_accuracy_score, roc_auc_score,
    precision_score, recall_score, f1_score,
    classification_report, confusion_matrix, roc_curve
)

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from imblearn.under_sampling import RandomUnderSampler, NeighbourhoodCleaningRule
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.exceptions import UndefinedMetricWarning

# Suppress warnings
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=UndefinedMetricWarning)

# Store feature names
feature_names = X.columns.tolist()

# Train-test split and scale
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Sampling strategies
sampling_strategies = {
    'Basemodel': None,
    'NCR': NeighbourhoodCleaningRule(),
    'RUS': RandomUnderSampler(random_state=42),
    'SMOTE': SMOTE(random_state=42)
}

# Models and hyperparameter grids
model_grid = {
    'LogisticRegression': {
        'model': LogisticRegression(max_iter=1000),
        'params': {'model__C': [0.01, 0.1, 1, 10]}
    },
    'DecisionTree': {
        'model': DecisionTreeClassifier(),
        'params': {'model__max_depth': [3, 5, 10]}
    },
    'RandomForest': {
        'model': RandomForestClassifier(),
        'params': {'model__n_estimators': [50, 100], 'model__max_depth': [5, 10]}
    },
    'XGBoost': {
        'model': XGBClassifier(eval_metric='logloss'),
        'params': {'model__n_estimators': [50, 100], 'model__max_depth': [3, 5]}
    }
}

# Storage
results = []
roc_data = {}
feature_importances = {}
shap_values_store = {}
conf_matrices = []
conf_titles = []

# SHAP config
shap_models = ['XGBoost']
shap_samplers = sampling_strategies.keys()

# Training and evaluation
for sampler_label, sampler in sampling_strategies.items():
    print(f"\n--- Running models with {sampler_label} data ---")

    if sampler:
        X_train_res, y_train_res = sampler.fit_resample(X_train, y_train)
    else:
        X_train_res, y_train_res = X_train.copy(), y_train.copy()

    print("y_train:", Counter(y_train_res))
    print("y_test :", Counter(y_test))
    print("X_train shape:", X_train_res.shape)
    print("X_test shape :", X_test.shape)

    for model_name, config in model_grid.items():
        try:
            print(f"Training {model_name} with {sampler_label} sampling...")
            pipeline = Pipeline([('model', config['model'])])
            grid = GridSearchCV(pipeline, config['params'], cv=3, scoring='roc_auc', n_jobs=-1)
            grid.fit(X_train_res, y_train_res)

            y_pred = grid.predict(X_test)
            y_proba = grid.predict_proba(X_test)[:, 1]

            # Store metrics + best params
            best_params = grid.best_params_
            results.append({
                'model': f"{model_name} ({sampler_label})",
                'accuracy': accuracy_score(y_test, y_pred),
                'balanced_accuracy': balanced_accuracy_score(y_test, y_pred),
                'roc_auc': roc_auc_score(y_test, y_proba),
                'precision': precision_score(y_test, y_pred, zero_division=0),
                'recall': recall_score(y_test, y_pred, zero_division=0),
                'f1_score': f1_score(y_test, y_pred, zero_division=0),
                'best_params': best_params
            })

            print(classification_report(y_test, y_pred))
            print(f"Best Params for {model_name} ({sampler_label}): {best_params}")

            # Confusion matrix
            cm = confusion_matrix(y_test, y_pred)
            conf_matrices.append(cm)
            conf_titles.append(f"{model_name} ({sampler_label})")

            # ROC curve
            fpr, tpr, _ = roc_curve(y_test, y_proba)
            roc_data[f"{model_name} ({sampler_label})"] = (fpr, tpr)

            # Feature importance
            base_model = grid.best_estimator_.named_steps['model']
            if hasattr(base_model, 'feature_importances_'):
                feature_importances[f"{model_name} ({sampler_label})"] = base_model.feature_importances_
            elif hasattr(base_model, 'coef_'):
                feature_importances[f"{model_name} ({sampler_label})"] = np.abs(base_model.coef_[0])
            else:
                print(f"No feature importance for {model_name} ({sampler_label})")

            # SHAP
            if model_name in shap_models and sampler_label in shap_samplers:
                print(f"Generating SHAP values for {model_name} ({sampler_label})...")
                explainer = shap.Explainer(base_model, X_train_res)
                shap_values = explainer(X_test)
                shap_values_store[f"{model_name} ({sampler_label})"] = shap_values

                shap.summary_plot(shap_values, X_test, feature_names=feature_names, show=False)
                plt.title(f"SHAP Summary - {model_name} ({sampler_label})")
                plt.tight_layout()
                plt.show()

        except Exception as e:
            print(f"Error training {model_name} ({sampler_label}): {e}")

# Convert results to DataFrame
results_df = pd.DataFrame(results)
print("\nModel Performance Summary:")
print(results_df[['model', 'accuracy', 'balanced_accuracy', 'roc_auc', 'best_params']])

In [ ]:
# --- ROC Curves with AUC Values in Legend ---
plt.figure(figsize=(12, 10))

# Build a lookup for AUC values
auc_lookup = dict(zip(results_df['model'], results_df['roc_auc']))

# Plot each ROC curve
for label, (fpr, tpr) in roc_data.items():
    auc = auc_lookup.get(label, None)
    label_with_auc = f"{label} (AUC = {auc:.3f})" if auc is not None else label
    plt.plot(fpr, tpr, label=label_with_auc)

# Plot baseline
plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier')
plt.title('ROC Curves for All Models and Sampling Techniques')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right', fontsize='small')
plt.grid(True)
plt.tight_layout()
plt.show()

# --- Feature Importance Plots for All Models and Sampling Techniques ---
for model_label, importances in feature_importances.items():
    try:
        # Convert to Series for better plotting
        imp_series = pd.Series(importances, index=feature_names)
        imp_series = imp_series.sort_values(ascending=False)[:10]  # Top 10 features

        plt.figure(figsize=(10, 6))
        sns.barplot(x=imp_series.values, y=imp_series.index)
        plt.title(f"Top 10 Feature Importances - {model_label}")
        plt.xlabel("Importance Score")
        plt.ylabel("Feature")
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"Error plotting feature importance for {model_label}: {e}")


# --- Accuracy and Balanced Accuracy Bar Charts ---
results_df_sorted = results_df.sort_values(by='model')

fig, axes = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# Accuracy
axes[0].bar(results_df_sorted['model'], results_df_sorted['accuracy'])
axes[0].set_title('Accuracy by Model and Sampling Strategy')
axes[0].set_ylabel('Accuracy')
axes[0].tick_params(axis='x', rotation=90)
for idx, value in enumerate(results_df_sorted['accuracy']):
    axes[0].text(idx, value + 0.01, f"{value:.1%}", ha='center', va='bottom', fontsize=8)

# Balanced Accuracy
axes[1].bar(results_df_sorted['model'], results_df_sorted['balanced_accuracy'])
axes[1].set_title('Balanced Accuracy by Model and Sampling Strategy')
axes[1].set_ylabel('Balanced Accuracy')
axes[1].tick_params(axis='x', rotation=90)
for idx, value in enumerate(results_df_sorted['balanced_accuracy']):
    axes[1].text(idx, value + 0.01, f"{value:.1%}", ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.show()

# --- Confusion Matrix Grid (4x4) ---
fig, axes = plt.subplots(4, 4, figsize=(18, 18))
fig.suptitle("Confusion Matrices for All Models and Sampling Strategies", fontsize=18)

for i, ax in enumerate(axes.flat):
    if i < len(conf_matrices):
        sns.heatmap(conf_matrices[i], annot=True, fmt='d', cmap='Blues', cbar=False,
                    xticklabels=['No Show = 0', 'No Show = 1'],
                    yticklabels=['No Show = 0', 'No Show = 1'], ax=ax)
        ax.set_title(conf_titles[i], fontsize=10)
        ax.set_xlabel("Predicted")
        ax.set_ylabel("Actual")
    else:
        ax.axis('off')

plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()

Accuracy, Balanced Accuracy, and AUC-ROC were selected as the key evaluation metrics because they provide a balanced and interpretable view of model performance, particularly in imbalanced classification tasks like healthcare predictions. Accuracy reflects the proportion of total correct predictions, including both true positives and true negatives. However, when one class dominates the dataset, accuracy alone can be misleading. Balanced Accuracy addresses this by averaging sensitivity (the model’s ability to correctly identify true positives) and specificity (its ability to correctly identify true negatives), offering a fairer reflection of performance across both classes (Guesné et al., 2024). AUC-ROC complements these by measuring how well the model separates positive from negative cases across all thresholds, providing a threshold-independent indicator of classification quality.


In [ ]:
# === Percentage Difference Compared to Basemodel (Core Metrics Only) ===
# Identify key metrics
results_df_renamed = results_df.rename(columns={
    'accuracy': 'Accuracy',
    'balanced_accuracy': 'Balanced Accuracy',
    'roc_auc': 'ROC-AUC Score'
})

# Extract model name and sampling strategy
results_df_renamed['Model'] = results_df_renamed['model'].apply(lambda x: x.split(' ')[0])
results_df_renamed['Sampling'] = results_df_renamed['model'].apply(lambda x: x.split('(')[-1].replace(')', ''))

# Select only core metrics
core_metrics = ['Accuracy', 'Balanced Accuracy', 'ROC-AUC Score']

# Split into baseline and comparison sets
base_df = results_df_renamed[results_df_renamed['Sampling'] == 'Basemodel'].set_index('Model')
comparison_df = results_df_renamed[results_df_renamed['Sampling'] != 'Basemodel'].set_index('Model')

# Join comparison with baseline metrics
comparison_with_base = comparison_df.join(
    base_df[core_metrics],
    lsuffix='_Current',
    rsuffix='_Basemodel',
    on='Model'
).reset_index()

# Compute % differences
for metric in core_metrics:
    current = f"{metric}_Current"
    base = f"{metric}_Basemodel"
    comparison_with_base[f"{metric} % Diff"] = (
        (comparison_with_base[current] - comparison_with_base[base]) / comparison_with_base[base]
    ) * 100

# Create summary table
percent_diff_cols = ['Model', 'Sampling'] + [f"{metric} % Diff" for metric in core_metrics]
percent_diff_df = comparison_with_base[percent_diff_cols]

# Display result
print("\n=== Percentage Difference Compared to Basemodel (Core Metrics Only) ===")
print(percent_diff_df.to_string(index=False))

# Prepare for heatmap
melted_pct = pd.melt(
    percent_diff_df,
    id_vars=['Model', 'Sampling'],
    var_name='Metric',
    value_name='Percentage Difference'
)

pivot_pct = melted_pct.pivot_table(index=['Model', 'Sampling'], columns='Metric', values='Percentage Difference')

# --- Clean up for heatmap ---
pivot_pct.replace([np.inf, -np.inf], np.nan, inplace=True)
pivot_pct.fillna(0, inplace=True)
pivot_pct = pivot_pct.clip(lower=-100, upper=100)

# Plot heatmap
plt.figure(figsize=(10, 2 + 0.5 * len(pivot_pct)))
sns.heatmap(pivot_pct, annot=True, fmt=".2f", cmap="coolwarm", center=0, linewidths=0.5)

# Title
strategies = results_df_renamed['Sampling'].unique().tolist()
if 'Basemodel' in strategies:
    strategies.remove('Basemodel')
strategy_title = " vs ".join(strategies)
plt.title(f"Percentage Difference (Accuracy, Balanced Accuracy, ROC-AUC) vs Basemodel ({strategy_title})")

plt.tight_layout()
plt.show()